In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

import time

#init display
from pynq.lib.arduino import Arduino_Displaycam
test = Arduino_Displaycam(base.ARDUINO)
test.init()
test.gpio()
test.pwm() 

0

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.arduino import Arduino_SPOT
device = Arduino_SPOT(base.ARDUINO,Arduino_SPOT)

In [3]:
device.clearWindow(1)

In [4]:
from pynq.lib.arduino.spot_device import *
import asyncio
import math

spot = SPOT(device)
spot.start()

# spot.device.setImageAddress()
# for i in range(0,1):
#     spot.device.snapPic(100,0)
    
#pickle
# savedList = spot.radar.savePicture(spot.device.drawAddr, "spottyThotty")
# print('pickled')


# #clear buffer
# for slot in spot.device.drawAddr:
#     slot = bytes([0])
# print('cleared buffer')

# device.clearAll()

# time.sleep(0.05)

# openList = []

# #unpickle
# openList = spot.radar.openPicture(spot.device.drawAddr, "spottyThotty")
# print('unpickled')

# #draw
# spot.device.drawImage(100,0)
# print('redrawn')

# print(savedList == openList)

# while(True):
#     print(spot.device.readFromGPS())
#     time.sleep(1)

import random
types = ['INTEREST', 'DANGER']
counter = 1
points = [Point('USER_{}'.format(i)) for i in range(0, 10)]
for i in points:
    i.tag = 'Obstacle_{}'.format(counter)
    i.createdBy = 'CB_{}'.format(counter)
    i.distance = 'DISTANCE_{}'.format(counter)
    i.type = types[random.randint(0, 1)]
    i.currentLocation = (random.randint(-30, 30), random.randint(-30, 30))
    counter += 1
    spot.radar.addPoint(i)

# #init test points to draw
# point0 = Point('Brandon')
# point0.tag = 'Hot'
# point0.createdBy = 'Beta Saurabh'
# point0.distance = 69
# point0.type = 'DANGER'
# point0.currentLocation = (20,0)

# point1 = Point('Saurabh')
# point1.tag = 'Weird'
# point1.createdBy = 'Alpha Brandon'
# point1.type = 'INTEREST'
# point1.distance = 420
# point1.currentLocation = (-15,15)


# print(len(spot.radar.points))
# spot.radar.addPoint(point0)
# spot.radar.addPoint(point1)
# print(len(spot.radar.points))

spot.radar.userLocation = (0,0)

#loop testing
# while(True):
#     spot.loop()
#     time.sleep(0.5)

In [5]:
'''
Types of events:
TOP
BOTTOM
RIGHT
LEFT
ERROR
'''

@asyncio.coroutine
def buttonPress(num):
    event = 'ERROR'
    twistCount = 0
    while True:
        yield from base.buttons[num].wait_for_value_async(1)
        if(num == 0):
            event = 'TOP'
            while base.buttons[num].read():
                base.leds[num].toggle()
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
        elif(num == 1):
            event = 'BOTTOM'
            while base.buttons[num].read():
                base.leds[num].toggle()
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
            
        else:
            event = 'ERROR'
        spot.on_event(event)
        
tasks = [asyncio.ensure_future(buttonPress(i)) for i in range(0,2)]

In [6]:
# '''
# self.write_TAG(100, 310)
# self.write_DISTANCE(100, 360)
# self.write_CB(100, 410)
# self.write_TYPE()
# '''

# device.clearAll()
# time.sleep(0.05)
# point = spot.radar.points[2]
# device.drawInfoState()
# if(point.type == 'DANGER'):
#     device.write_CUSTOM(point.type, 200, 265, 0xf800) # Variable type
# else:
#     device.write_CUSTOM(point.type, 200, 265, 0x07e8) # Variable type
# device.write_CUSTOM(point.tag, 190, 315, 0xffff) # Variable tag
# device.write_CUSTOM(str(point.distance) + ' meters', 270, 365, 0xffff) # Variable distance
# device.write_CUSTOM(point.createdBy, 300, 415, 0xffff) # Variable c/b

In [7]:
def readUntil(val, delim):
    returnList = []
    for x in val:
        if(x == delim):
            return returnList
        returnList.append(x)

@asyncio.coroutine
def txRead():
    pointID = None
    objectType = None
    tag = None
    createdBy = None
    lat = None
    long = None
    cameraList = []
    while(True):
        yield from asyncio.sleep(0.1)
        while(device.hasMessages()):
            val = device.readFromTX()
            val = [chr(x) for x in val]
#             print(val)
            if(val[3] == 'i'): # Point ID
                val = readUntil(val, "!")[5:]
                pointID = int(''.join(val))
            elif(val[3] == 'y'): # Object Type
                val = readUntil(val, "!")[5:]
                objectType = ''.join(val)
            elif(val[3] == 't'): # Tag
                val = readUntil(val, "!")[5:]
                tag = ''.join(val)
            elif(val[3] == 'b'): # Tag
                val = readUntil(val, "!")[5:]
                createdBy = ''.join(val)
            elif(val[3] == 'g'): # LAT / LONG
                val = ''.join(readUntil(val, "!")[5:]).split(",")
                lat = float(''.join(val[0]))
                long = float(''.join(val[1]))
            elif(val[3] == 'c'): # CAMERA
                val = [ord(x) for x in val]
                cameraList += val[4:]
            elif(val[3] == 'd'): # DONE
                cameraList = cameraList[:76800]
                print(pointID, objectType, tag, lat, long, len(cameraList))
#                 spot.radar.createPoint(pointID, tag, createdBy, (lat,long), objectType, cameraList)
#                 print(cameraList)
                spot.radar.createPoint(pointID, tag, createdBy, (lat, long), objectType, cameraList)
                print('created point')
                ## Create new Point
                pointID = 0
                objectType = None
                createdBy = None
                tag = None
                lat = None
                long = None
                cameraList = []

tasks.append(asyncio.ensure_future(txRead()))

In [8]:
@asyncio.coroutine
def loop():
    while True:
        yield from asyncio.sleep(0.3)
        spot.loop()

tasks.append(asyncio.ensure_future(loop()))

In [9]:
# @asyncio.coroutine
# def gpsSend():
#     while True:
#         yield from asyncio.sleep(3)
#         gpsLoc = device.readFromGPS()
#         if(gpsLoc[0] != None):
#             gpsString = "g," + str(round(gpsLoc[0], 6)) + "," + str(round(gpsLoc[1], 6))
#             print(gpsString)
#             device.writeToTX(4, gpsString)
#             spot.radar.userLocation = tuple(111111*math.cos(math.radians(gpsLoc[0])), 111111*gpsLoc[1])
# tasks.append(asyncio.ensure_future(gpsSend()))

In [10]:
# import psutil

# @asyncio.coroutine
# def print_cpu_usage():
#     # Calculate the CPU utilisation by the amount of idle time
#     # each CPU has had in three second intervals
#     last_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
#     while True:
#         yield from asyncio.sleep(3)
#         next_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
#         usage = [(1-(c2-c1)/3) * 100 for c1,c2 in zip(last_idle, next_idle)]
#         print("CPU Usage: {0:3.2f}%, {1:3.2f}%".format(*usage))
#         last_idle = next_idle

# tasks.append(asyncio.ensure_future(print_cpu_usage()))

In [11]:
if base.switches[0].read():
    print("Please set switch 0 low before running")
else:
#     spot.loop()
    base.switches[0].wait_for_value(1)

KeyboardInterrupt: 

In [ ]:
#unpickle
val = spot.radar.openPicture(spot.device.drawAddr, "3")
# print(val)
# for index in range(76800):
#     print(ord(spot.device.drawAddr[index]))
print('unpickled')
#draw
spot.device.drawImage(100,0)
print('redrawn')
